In [28]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Data Load

In [29]:
from keras.datasets import cifar100

def get_cifar100():
    (x_train, y_train), (x_test, y_test) = cifar100.load_data()
    return x_train, y_train, x_test, y_test

In [30]:
x_data, y_data, x_test, y_test = get_cifar100()

In [31]:
from sklearn.model_selection import train_test_split

x_train, x_val, y_train, y_val = train_test_split(x_data, y_data, test_size=0.2, shuffle=True, random_state=42)

## Data Preprocess

In [32]:
from tensorflow.python.util.tf_export import keras_export
from keras.applications import imagenet_utils

@keras_export("keras.applications.vgg16.preprocess_input")
def preprocess_input(x, data_format=None):
    return imagenet_utils.preprocess_input(
        x, data_format=data_format, mode="caffe"
    )

In [33]:
def data_augmentation(data):
  x = tf.keras.layers.Rescaling(1./255)(data)
  x=tf.keras.layers.RandomFlip('horizontal')(x)
  return x

In [34]:
# prex_train = data_augmentation(x_train)
# prex_val = data_augmentation(x_val)

# prex_train = preprocess_input(prex_train)
# prex_val = preprocess_input(prex_val)
# prex_test = preprocess_input(x_test)

In [35]:
prex_train = preprocess_input(x_train)
prex_val = preprocess_input(x_val)
prex_test = preprocess_input(x_test)

In [36]:
import numpy as np
class_num = len(np.unique(y_train.reshape(-1,)))

In [37]:
from tensorflow.keras.utils import to_categorical

prey_train = to_categorical(y_train, class_num)
prey_val = to_categorical(y_val, class_num)
prey_test = to_categorical(y_test, class_num)

In [38]:
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf

batch_size = 256
img_size = (32, 32)

In [39]:
# tf.data.Dataset.from_tensor_slices((prex_train, prey_train)).cardinality().numpy()

In [40]:
train_dataset = tf.data.Dataset.from_tensor_slices((prex_train, prey_train))
train_dataset = train_dataset.shuffle(buffer_size=40000) 
train_dataset = train_dataset.batch(batch_size)
train_dataset = train_dataset.prefetch(tf.data.experimental.AUTOTUNE)

In [41]:
val_dataset = tf.data.Dataset.from_tensor_slices((prex_val, prey_val))
val_dataset = val_dataset.shuffle(buffer_size=10000)
val_dataset = val_dataset.batch(batch_size)
val_dataset = val_dataset.prefetch(tf.data.experimental.AUTOTUNE)

In [42]:
test_dataset = tf.data.Dataset.from_tensor_slices((prex_test, prey_test))
test_dataset = test_dataset.shuffle(buffer_size=10000)
test_dataset = test_dataset.batch(batch_size)
test_dataset = test_dataset.prefetch(tf.data.experimental.AUTOTUNE)

In [43]:
# class_names = np.unique(y_train.reshape(-1,))

# plt.figure(figsize=(10, 10))
# for images, labels in train_dataset.take(1):
#   for i in range(9):
#     ax = plt.subplot(3, 3, i + 1)
#     plt.imshow(images[i].numpy().astype("uint8"))
#     plt.title(class_names[labels[i]])
#     plt.axis("off")

In [44]:
# for image, _ in train_dataset.take(1):
#   plt.figure(figsize=(10, 10))
#   first_image = image[0]
#   for i in range(9):
#     ax = plt.subplot(3, 3, i + 1)
#     augmented_image = data_augmentation(tf.expand_dims(first_image, 0))
#     plt.imshow(augmented_image[0] / 255)
#     plt.axis('off')

## Load VGG16 Model

In [45]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input
import numpy as np

# vgg_model = VGG16(weights='imagenet', include_top=False)

IMG_SHAPE = img_size + (3,)
vgg_model = tf.keras.applications.vgg16.VGG16(
    include_top = False,
    weights='imagenet',
    input_tensor=None,
    input_shape=IMG_SHAPE,
    pooling=None,
    classes=1000,
    classifier_activation='softmax'
)

In [46]:
vgg_model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 32, 32, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 32, 32, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 16, 16, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 16, 16, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 16, 16, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 8, 8, 128)         0     

# Transfer learning

In [47]:
for layer in vgg_model.layers:
  layer.trainable =False

In [48]:
# from keras.models import Sequential
# from tensorflow.keras import layers

# model = Sequential()
# model.add(vgg_model)
# model.add(layers.GlobalAveragePooling2D())
# model.add(layers.Dense(32*32*2, activation='relu'))
# # model.add(layers.Dense(32, activation='relu'))
# model.add(layers.Dense(32*32, activation='relu'))
# model.add(layers.Dense(class_num, activation='softmax'))

In [49]:
from keras.models import Sequential
from tensorflow.keras import layers

model = Sequential()
model.add(vgg_model)
model.add(layers.Flatten())
# model.add(layers.Dense(32*32*4, activation='relu'))
# model.add(layers.Dropout(0.5))
model.add(layers.Dense(32*32*2, activation='relu'))
model.add(layers.Dense(32*32*2, activation='relu'))
# model.add(layers.Dropout(0.5))
# model.add(layers.Dense(32, activation='relu'))
# model.add(layers.Dense(32*32, activation='relu'))
# model.add(layers.Dropout(0.5))
model.add(layers.Dense(class_num, activation='softmax'))

In [50]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 1, 1, 512)         14714688  
                                                                 
 flatten_1 (Flatten)         (None, 512)               0         
                                                                 
 dense_2 (Dense)             (None, 2048)              1050624   
                                                                 
 dense_3 (Dense)             (None, 2048)              4196352   
                                                                 
 dense_4 (Dense)             (None, 100)               204900    
                                                                 
Total params: 20,166,564
Trainable params: 5,451,876
Non-trainable params: 14,714,688
_________________________________________________________________


In [51]:
from tensorflow.keras.optimizers import Adam

model.compile(loss='categorical_crossentropy',
              optimizer=Adam(learning_rate=1e-3),
              metrics=['acc'])

In [52]:
import datetime
from keras import callbacks

dir_name = "Learning_log"

def make_Tensorboard_dir(dir_name):
  root_logdir = os.path.join("/content/drive/MyDrive", dir_name)
  sub_dir_name = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
  return os.path.join(root_logdir, sub_dir_name)

TB_log_dir = make_Tensorboard_dir(dir_name)
Tensor_board = callbacks.TensorBoard(log_dir = TB_log_dir)
lr_cb = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.01, patience=3, mode='min', verbose=1)

# log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
# tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
early_stop = callbacks.EarlyStopping(monitor="val_loss",
                                           min_delta=0,
                                           patience=5,
                                           restore_best_weights=True)


In [53]:
tf.config.run_functions_eagerly(True)
history = model.fit(train_dataset, epochs = 50, 
                    validation_data=(val_dataset),
                    callbacks=[early_stop, Tensor_board, lr_cb],
                    validation_steps=4)

Epoch 1/50
 20/157 [==>...........................] - ETA: 1:12:32 - loss: 13.3839 - acc: 0.1094

KeyboardInterrupt: ignored

In [ ]:
# 모델 저장
# model.save("/content/drive/MyDrive/Learning_log")

## Accuracy Graph

In [ ]:
# import matplotlib.pyplot as plt

# acc = history.history['acc']
# loss = history.history['loss']
 
# epochs = range(len(acc))
 
# plt.plot(epochs, acc, 'b', label='accuracy')
# plt.plot(epochs, loss, 'r', label='loss')
# plt.title('accuracy and loss')
# plt.legend()
  
# plt.show()

In [ ]:
# import matplotlib.pyplot as plt

# acc = history.history['acc']
# loss = history.history['loss']
 
# epochs = range(len(acc))
 
# plt.plot(epochs, acc, 'b', label='accuracy')
# plt.plot(epochs, loss, 'r', label='loss')
# plt.title('accuracy and loss')
# plt.legend()
  
# plt.show()

In [ ]:
loss, accuracy = model.evaluate(test_dataset)
print('Test accuracy :', accuracy)

In [ ]:
# !pip install tensorboardX 

In [ ]:
# %load_ext tensorboard

In [ ]:
%tensorboard --logdir /content/drive/MyDrive/Learning_log
